In [37]:
import pandas as pd
import numpy as np
import torch
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch import Trainer
from lightning.pytorch import loggers as pl_loggers

from library.taxonomy_constructors import SyntheticTaxonomy, CrossPredictionsTaxonomy
from library.models import ResNetModel
from library.datasets import CIFAR100MappedDataModule, CIFAR100Mapped

# Load CIFAR100 dataset information
cifar100_dataset = CIFAR100Mapped(root="datasets/cifar100", download=False)
cifar100_labels = cifar100_dataset.classes
print(f"Total CIFAR100 classes: {len(cifar100_labels)}")

# Create a simple target mapping for CIFAR100 (0 to num_classes-1)
cifar100_targets = pd.DataFrame(
    {"class_id": range(len(cifar100_labels)), "class_name": cifar100_labels}
)

# Reduce the precision of matrix multiplication to speed up training
torch.set_float32_matmul_precision("medium")

Total CIFAR100 classes: 100


In [38]:
# Create synthetic taxonomy with 2 domains for CIFAR100
synthetic_taxonomy = SyntheticTaxonomy.create_synthetic_taxonomy(
    num_atomic_concepts=len(cifar100_labels),
    num_domains=2,
    domain_class_count_mean=50,
    domain_class_count_variance=5,
    concept_cluster_size_mean=3,
    concept_cluster_size_variance=1,
    no_prediction_class=True,
    atomic_concept_labels=cifar100_labels,
    relationship_type="true",
)

# Extract domain mappings
domain_A_mapping = synthetic_taxonomy.domains[0].to_mapping()
domain_B_mapping = synthetic_taxonomy.domains[1].to_mapping()

print(f"Domain A classes: {len(set(domain_A_mapping.values()))}")
print(f"Domain B classes: {len(set(domain_B_mapping.values()))}")
print(f"Domain A original classes: {len(domain_A_mapping)}")
print(f"Domain B original classes: {len(domain_B_mapping)}")

Domain A classes: 17
Domain B classes: 17
Domain A original classes: 52
Domain B original classes: 50


In [39]:
# Configuration
TRAIN = False


def train_domain_model(domain_mapping, domain_name, logger_name, model_name):
    """Train a ResNet model for a specific domain using CIFAR100 with overfitting mitigation"""
    tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs", name=logger_name)
    dataset = CIFAR100MappedDataModule(mapping=domain_mapping, batch_size=256)

    callbacks = [
        ModelCheckpoint(
            dirpath="checkpoints",
            monitor="val_loss",
            mode="min",
            save_top_k=1,
            filename=model_name,
            enable_version_counter=False,
        ),
        LearningRateMonitor(logging_interval="epoch"),
    ]

    trainer = Trainer(
        max_epochs=100,
        logger=tb_logger if TRAIN else False,
        callbacks=callbacks,
        gradient_clip_val=1.0,
        gradient_clip_algorithm="norm",
    )

    if TRAIN:
        model = ResNetModel(
            num_classes=len(set(domain_mapping.values())),
            architecture="resnet50",
            optim="adamw",
            optim_kwargs={
                "lr": 0.001,
                "weight_decay": 1e-3,
            },
            # Add learning rate scheduling
            lr_scheduler="multistep",
            lr_scheduler_kwargs={
                "milestones": [30, 60, 80],
                "gamma": 0.1,
            },
        )
        trainer.fit(model, datamodule=dataset)
        results = trainer.test(datamodule=dataset, ckpt_path="best")
    else:
        model = ResNetModel.load_from_checkpoint(f"checkpoints/{model_name}.ckpt")
        results = trainer.test(model, datamodule=dataset)

    print(f"{domain_name} Results: {results}")
    return results

In [40]:
print("Training/Testing Domain A Model:")
domain_A_results = train_domain_model(
    domain_A_mapping,
    "Domain A",
    "cifar100_synthetic_A",
    "resnet50-cifar100-synthetic-A-min-val-loss",
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Training/Testing Domain A Model:


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 21/21 [00:01<00:00, 18.25it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.7674999833106995
        eval_loss           0.9303333163261414
        hp_metric           0.7674999833106995
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Domain A Results: [{'eval_loss': 0.9303333163261414, 'eval_accuracy': 0.7674999833106995, 'hp_metric': 0.7674999833106995}]


In [41]:
# Train Domain B model
print("Training/Testing Domain B Model:")
domain_B_results = train_domain_model(
    domain_B_mapping,
    "Domain B",
    "cifar100_synthetic_B",
    "resnet50-cifar100-synthetic-B-min-val-loss",
)

Training/Testing Domain B Model:


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 20/20 [00:01<00:00, 17.92it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.7487999796867371
        eval_loss           0.9866886734962463
        hp_metric           0.7487999796867371
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Domain B Results: [{'eval_loss': 0.9866886734962463, 'eval_accuracy': 0.7487999796867371, 'hp_metric': 0.7487999796867371}]


In [42]:
# Configuration for prediction generation
PREDICT = False

if PREDICT:
    # Load datasets
    dataset_domain_A = CIFAR100MappedDataModule(
        mapping=domain_A_mapping, batch_size=128
    )
    dataset_domain_B = CIFAR100MappedDataModule(
        mapping=domain_B_mapping, batch_size=128
    )

    # Load trained models
    model_domain_A = ResNetModel.load_from_checkpoint(
        "checkpoints/resnet50-cifar100-synthetic-A-min-val-loss.ckpt"
    )
    model_domain_A.eval()

    model_domain_B = ResNetModel.load_from_checkpoint(
        "checkpoints/resnet50-cifar100-synthetic-B-min-val-loss.ckpt"
    )
    model_domain_B.eval()

    trainer = Trainer(logger=False, enable_checkpointing=False)

    # Generate cross-domain predictions
    print("Generating cross-domain predictions...")
    model_A_on_domain_B = trainer.predict(model_domain_A, datamodule=dataset_domain_B)
    model_B_on_domain_A = trainer.predict(model_domain_B, datamodule=dataset_domain_A)

    # Convert predictions to class indices
    predictions_A_on_B = torch.cat(model_A_on_domain_B).argmax(dim=1)  # type: ignore
    predictions_B_on_A = torch.cat(model_B_on_domain_A).argmax(dim=1)  # type: ignore

    # Get ground truth targets
    domain_A_targets = torch.cat(
        [label for _, label in dataset_domain_A.predict_dataloader()]
    )
    domain_B_targets = torch.cat(
        [label for _, label in dataset_domain_B.predict_dataloader()]
    )

    # Save predictions
    pd.DataFrame(
        {
            "domain_A": domain_A_targets,
            "predictions_B_on_A": predictions_B_on_A,
        }
    ).to_csv("data/cifar100_2domain_A_predictions.csv", index=False)

    pd.DataFrame(
        {
            "domain_B": domain_B_targets,
            "predictions_A_on_B": predictions_A_on_B,
        }
    ).to_csv("data/cifar100_2domain_B_predictions.csv", index=False)

    print("Predictions saved to CSV files.")

# Load prediction results
df_A = pd.read_csv("data/cifar100_2domain_A_predictions.csv")
df_B = pd.read_csv("data/cifar100_2domain_B_predictions.csv")

print(f"Domain A predictions shape: {df_A.shape}")
print(f"Domain B predictions shape: {df_B.shape}")

Domain A predictions shape: (5200, 2)
Domain B predictions shape: (5000, 2)


In [43]:
# Construct taxonomy from cross-domain predictions
taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
    cross_domain_predictions=[
        (0, 1, np.array(df_B["predictions_A_on_B"], dtype=np.intp)),
        (1, 0, np.array(df_A["predictions_B_on_A"], dtype=np.intp)),
    ],
    domain_targets=[
        (0, np.array(df_A["domain_A"], dtype=np.intp)),
        (1, np.array(df_B["domain_B"], dtype=np.intp)),
    ],
    domain_labels=synthetic_taxonomy.domain_labels,
    relationship_type="density_threshold",
    threshold=0.6,
)

print("Taxonomy constructed from cross-domain predictions.")

Taxonomy constructed from cross-domain predictions.


In [44]:
# Generate and save taxonomy visualizations
print("Generating taxonomy visualizations...")

taxonomy.visualize_graph("2-Domain CIFAR100 Model Taxonomy").save_graph(
    "output/cifar100_2domain_synthetic_model_taxonomy.html"
)

synthetic_taxonomy.visualize_graph(
    "2-Domain CIFAR100 Ground Truth Taxonomy"
).save_graph("output/cifar100_2domain_synthetic_taxonomy.html")

print("Taxonomy visualizations saved to output/ directory.")

Generating taxonomy visualizations...
Taxonomy visualizations saved to output/ directory.


In [45]:
# Evaluate taxonomy against ground truth
edr = taxonomy.edge_difference_ratio(synthetic_taxonomy)
precision, recall, f1 = taxonomy.precision_recall_f1(synthetic_taxonomy)

print("2-Domain CIFAR100 Taxonomy Evaluation:")
print(f"Edge Difference Ratio: {edr:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

2-Domain CIFAR100 Taxonomy Evaluation:
Edge Difference Ratio: 0.5993
Precision: 0.3884
Recall: 0.8704
F1 Score: 0.5371


In [46]:
# Build universal taxonomies
print("Building universal taxonomies...")

taxonomy.build_universal_taxonomy()
taxonomy.visualize_graph("2-Domain CIFAR100 Model Universal Taxonomy").save_graph(
    "output/cifar100_2domain_synthetic_model_universal_taxonomy.html"
)

synthetic_taxonomy.build_universal_taxonomy()
synthetic_taxonomy.visualize_graph(
    "2-Domain CIFAR100 Ground Truth Universal Taxonomy"
).save_graph("output/cifar100_2domain_synthetic_universal_taxonomy.html")

print("Universal taxonomy visualizations saved to output/ directory.")

Building universal taxonomies...


Universal taxonomy visualizations saved to output/ directory.
